In [22]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

from fuzzywuzzy import fuzz as fu
from fuzzywuzzy import process
from rapidfuzz import fuzz,process, utils


import jellyfish
import snowflake.connector
# import torch
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('stsb-roberta-large')

# Imported the inaugural corpus from nltk.corpus
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import gensim
from gensim.utils import simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import num2words
from num2words import num2words
import time


pd.options.display.max_colwidth = 10000
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')


In [2]:
cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_XS', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')


In [3]:
con = cnx.cursor()

In [4]:
query = '''SELECT * FROM (
SELECT DISTINCT p9.PRODUCT_MASTER_CATALOG_KEY, i9.INVOICE_UPC_CASE,i9.INVOICE_PACKAGE_DESCRIPTION, p9.PRODUCT_DESCRIPTION, p9.GTIN_12_UPC, p9.GTIN
    FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE i9
    JOIN PRODUCT_MASTER_CATALOG p9 on RIGHT(p9.GTIN_12_UPC,10) = i9.INVOICE_UPC_CASE
    WHERE i9.INVOICE_DATE >= '2020-01-01' and LEN(i9.INVOICE_UPC_CASE) = 10 and i9.INVOICE_ITEM_TOTAL_PRICE > 0
        and i9.INVOICE_PACKAGE_DESCRIPTION not in (SELECT INVOICE_PACKAGE_DESCRIPTION FROM INVOICE_NONPRODUCT_FILTER)
    UNION
    SELECT DISTINCT p10.PRODUCT_MASTER_CATALOG_KEY, i10.INVOICE_UPC_CASE, i10.INVOICE_PACKAGE_DESCRIPTION, p10.PRODUCT_DESCRIPTION, p10.GTIN_12_UPC, p10.GTIN
    FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE i10
    JOIN PRODUCT_MASTER_CATALOG p10 on LEFT(p10.GTIN_12_UPC,10) = i10.INVOICE_UPC_CASE
    WHERE i10.INVOICE_DATE >= '2020-01-01' and LEN(i10.INVOICE_UPC_CASE) = 10 and i10.INVOICE_ITEM_TOTAL_PRICE > 0
        and i10.INVOICE_PACKAGE_DESCRIPTION not in (SELECT INVOICE_PACKAGE_DESCRIPTION FROM INVOICE_NONPRODUCT_FILTER)
  )
  ORDER BY INVOICE_UPC_CASE'''

In [5]:
results = con.execute(query)

In [6]:
res = pd.DataFrame.from_records(iter(results), columns=[y[0] for y in results.description])


In [7]:
len(res)

4475

In [9]:
res['MATCHING_SCORE'] = res.apply(lambda x : fu.partial_ratio(x.INVOICE_PACKAGE_DESCRIPTION, x.PRODUCT_DESCRIPTION), axis = 1)

In [10]:
res['CountOfUPCCASE'] = res.groupby('INVOICE_UPC_CASE')['INVOICE_UPC_CASE'].transform('count')

In [11]:
rx = res[["PRODUCT_MASTER_CATALOG_KEY","INVOICE_UPC_CASE","CountOfUPCCASE","INVOICE_PACKAGE_DESCRIPTION","PRODUCT_DESCRIPTION","MATCHING_SCORE","GTIN_12_UPC","GTIN"]]

In [12]:
rxs = res.sort_values(["INVOICE_UPC_CASE","CountOfUPCCASE","MATCHING_SCORE"], ascending=False).groupby(['INVOICE_UPC_CASE','GTIN']).head(1)

In [13]:
rxs.sort_values("INVOICE_UPC_CASE").head(50)

,PRODUCT_MASTER_CATALOG_KEY,INVOICE_UPC_CASE,INVOICE_PACKAGE_DESCRIPTION,PRODUCT_DESCRIPTION,GTIN_12_UPC,GTIN,MATCHING_SCORE,CountOfUPCCASE
0,9007,0000183147,RANCHO LA GLORIA PEACH MA 12/750ML,RANCHO LA GLORIA PEACH 13.9% 750ML GLASS,860000183147,00860000183147,76,1
2,9058,0000183154,UNO MAS RTS WTRMLN MARG 6/1.5L,UNO MAS WATERMELON 1.5L 13.9%,860000183154,00860000183154,55,2
3,9060,0000183178,UNO MAS LIME BUBBLES CAN 24/250ML,UNO MAS RTD REGULAR 250ML 9.9% 4-PACK,860000183178,00860000183178,43,1
4,9061,0000183185,UNO MAS BUBB STRB LIME CA 24/250ML,UNO MAS RTD STRAWBERRY 250ML 9.9%,860000183185,00860000183185,45,1
5,114888,0000186803,LOLO AND BO ROSE 12/750ML,LOLA AND BO ROSE,860000186803,00860000186803,94,1
6,114889,0000186810,BONOBO CHARDONNAY SELECT 12/750ML,BONOBO WINERY CHARDONNAY SELECT,860000186810,00860000186810,77,1
7,114894,0000186865,BONOBO PINOT NOIR 12/750ML,BONOBO WINERY PINOT NOIR,860000186865,00860000186865,71,1
8,9063,0000557603,UNO MAS RTD BUBB PCH MARG 24/250ML,UNO MAS RTD PEACH 250ML 9.9%,860000557603,00860000557603,64,1
9,9068,0000557658,UNO MAS RTD BUBB WTRMLN M 24/250ML,UNO MAS RTD WATERMELON 250ML 9.9%,860000557658,00860000557658,61,1
10,9008,0000557672,RNHCO GLRIA MARG BUBBLES 24/250ML,RANCHO LA GLORIA 250ML 9.9%,860000557672,00860000557672,52,1


In [20]:
len(rxs)

2839

In [14]:
rxs.to_csv("~/Downloads/revised_scored_len10_fuzzymatch.csv")

In [19]:
len(rxs['INVOICE_UPC_CASE']  != rxs['GTIN'])

2839

In [ ]:
# rxs.to_csv('~/Downloads/scored_len11_fuzzymatch.csv')

In [ ]:
# partial_ratio = []

# for i in r.itertuples():
#     innerstart = time.time()
#     invoice = i.INVOICE_PACKAGE_DESCRIPTION
#     query = i.PRODUCT_DESCRIPTION
#     partial_ratio.append([i.PRODUCT_MASTER_CATALOG_KEY,i.INVOICE_UPC_CASE,i.INVOICE_PACKAGE_DESCRIPTION, fuzz.partial_ratio(i.INVOICE_PACKAGE_DESCRIPTION,query), i.PRODUCT_DESCRIPTION])
# #         token_set_ratio.append([i.INVOICE_PACKAGE_DESCRIPTION, fuzz.token_set_ratio(i.INVOICE_PACKAGE_DESCRIPTION,query), p.PRODUCT_DESCRIPTION])
#     inner_finish = time.time() - innerstart
# print(i.INVOICE_PACKAGE_DESCRIPTION,inner_finish)


# # token_set_ratio = pd.DataFrame(token_set_ratio, columns=('attribute','set_score','target')).sort_values('set_score',ascending=False).reset_index()

In [ ]:
len(res)

In [23]:
start = time.time()
res['rapidfuzzy_MATCHING_SCORE'] = res.apply(lambda x : fuzz.partial_ratio(x.INVOICE_PACKAGE_DESCRIPTION, x.PRODUCT_DESCRIPTION), axis = 1)
print("Rapid FuzzyWuzzy Time", (time.time() - start))

Rapid FuzzyWuzzy Time 0.16961669921875


In [24]:
start = time.time()
res['fuzzy_MATCHING_SCORE'] = res.apply(lambda x : fu.partial_ratio(x.INVOICE_PACKAGE_DESCRIPTION, x.PRODUCT_DESCRIPTION), axis = 1)
print("FuzzyWuzzy Time", (time.time() - start))

FuzzyWuzzy Time 2.8479766845703125


In [25]:
res['diff'] = ((res['rapidfuzzy_MATCHING_SCORE']-res['fuzzy_MATCHING_SCORE'])/(res['fuzzy_MATCHING_SCORE']))*(100)

In [28]:
res[['rapidfuzzy_MATCHING_SCORE','fuzzy_MATCHING_SCORE']] 

,rapidfuzzy_MATCHING_SCORE,fuzzy_MATCHING_SCORE
0,76.315789,76
1,48.275862,48
2,55.172414,55
3,45.945946,43
4,48.484848,45
5,93.750000,94
6,77.419355,77
7,70.833333,71
8,64.285714,64
9,63.636364,61
